In [1]:
import numpy  as np
import pandas as pd
import json
import urllib 
import time
import datetime

from datetime import datetime

from bs4 import BeautifulSoup

import requests 
import re

from selenium import webdriver

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from selenium.webdriver.chrome.options import Options 

In [65]:
#path = 'chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
# chrome_options = Options() 
# headless 설정 
chrome_options.headless = False
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
caps = DesiredCapabilities().CHROME 
caps["pageLoadStrategy"] = None
driver = webdriver.Chrome("chromedriver", options=chrome_options)
driver.implicitly_wait(5)  ## 웹 자원을 3초 기다리기

In [3]:
ignore = re.compile('[\n\r\t\xa0\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U0001F90D-\U0001F9FF\u202f\xe1\u119e\u30fc\u200b\u2764\ufe0f\u1ef3\xeb\xe9\u1ea7\xc1\U000e0067\U000e0062\U000e0077\U000e006c\U000e0073\U000e007f\u8fdc\u8c13]')
def ignore_text(text):
    return ignore.sub(' ', text).strip()

In [61]:
travel_records = pd.DataFrame(data=[], columns=['place_name', 'user', 'travel_date', 'point', 'review'])

In [5]:
# urls=[
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d320364-Reviews-or0-Bulguksa_Temple-Gyeongju_Gyeongsangbuk_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g946498-d2478921-Reviews-or0-Busosanseong_Fortress-Buyeo_gun_Chungcheongnam_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g946498-d2478921-Reviews-or0-Busosanseong_Fortress-Buyeo_gun_Chungcheongnam_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g946498-d1993587-Reviews-or0-Baekje_History_Reproduction_Complex-Buyeo_gun_Chungcheongnam_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g946498-d11626760-Reviews-or0-Gungnamji_Pond-Buyeo_gun_Chungcheongnam_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2023540-d18188634-Reviews-or0-Oseosan-Hongseong_gun_Chungcheongnam_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2023538-d10388622-Reviews-or0-Jeonghyesa-Cheongyang_gun_Chungcheongnam_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2023538-d19881149-Reviews-or0-Chilgapsan_Provincial_Park-Cheongyang_gun_Chungcheongnam_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2023538-d19881206-Reviews-or0-Alps_Village-Cheongyang_gun_Chungcheongnam_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2018516-d11695367-Reviews-or0-Hongcheon_Ginkgo_Forest-Hongcheon_gun_Gangwon_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2018519-d15699380-Reviews-or0-Cheonjugyo_Pungsuwon_Catholic_Church-Hoengseong_gun_Gangwon_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2018519-d15075990-Reviews-or0-Taegi_San-Hoengseong_gun_Gangwon_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2018519-d15079083-Reviews-or0-Hoengseong_Hosu_Gil-Hoengseong_gun_Gangwon_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2018511-d14450689-Reviews-or0-Seondol_Cliff-Yeongwol_gun_Gangwon_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2024849-d21264693-Reviews-or0-Mungyeong_Saejae_Provincial_Park-Mungyeong_Gyeongsangbuk_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2024849-d18919789-Reviews-or0-Mungyeong_Omija_Theme_Tunnel-Mungyeong_Gyeongsangbuk_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g2024849-d3805641-Reviews-or0-Daeseungsa_Temple-Mungyeong_Gyeongsangbuk_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g1960566-d21302217-Reviews-or0-Gangcheon_Island-Yeoju_si_Gyeonggi_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g1960566-d3423548-Reviews-or0-Silleuksa_Temple-Yeoju_si_Gyeonggi_do.html',
#       'https://www.tripadvisor.co.kr/Attraction_Review-g1138011-d17310293-Reviews-or0-Yongmunsan_Resort-Yangpyeong_gun_Gyeonggi_do.html',
#     ]
urls = [
#     'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d324888-Reviews-or0-Gyeongbokgung_Palace-Seoul.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024844-d14008059-Reviews-or0-Seryangje_Reservoir-Hwasun_gun_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d1554895-Reviews-or0-Donggung_Palace_and_Wolji_Pond-Gyeongju_Gyeongsangbuk_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297885-d4504781-Reviews-or0-Hyeopjae_Beach-Jeju_Jeju_Island.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024861-d8055570-Reviews-or0-Cheongdo_Bullfighting_Stadium-Cheongdo_gun_Gyeongsangbuk_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g612377-d6116363-Reviews-or0-Daegwanryeong_Yangtte_Ranch-Pyeongchang_gun_Gangwon_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024833-d3399689-Reviews-or0-Boseong_Green_Tea_Field_Daehan_Dawon-Boseong_gun_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g612376-d3808249-Reviews-or0-Yeojwacheon_Stream-Changwon_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074107-d3209597-Reviews-or0-Nagan_Eupseong_Folk_Village-Suncheon_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024831-d2546638-Reviews-or0-Damyang_Juknokwon-Damyang_gun_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2368549-d3808282-Reviews-or0-Jirisan_Cheonwangbong_Peak-Sancheong_gun_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2150160-d320367-Reviews-or0-Haeinsa_Temple-Hapcheon_Gun_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297885-d1776326-Reviews-or0-Udo-Jeju_Jeju_Island.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297892-d2337355-Reviews-or0-Seopjikoji-Seogwipo_Jeju_Island.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2150165-d3823693-Reviews-or0-Upo_Wetland-Changnyeong_gun_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g612376-d8285449-Reviews-or0-Gyeonghwa_Station_Cherry_Blossom_Road-Changwon_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d4354720-Reviews-or0-Gwangandaegyo_Bridge-Busan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2023539-d8673288-Reviews-or0-Kkotji_Beach-Taean_gun_Chungcheongnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297892-d1582693-Reviews-or0-Seongsan_Ilchulbong-Seogwipo_Jeju_Island.html',
]
# driver.get(urls[1])

In [6]:
urls = [
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024866-d3164771-Reviews-or0-National_Gwacheon_Science_Museum-Gwacheon_Gyeonggi_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024866-d2041648-Reviews-or0-Seoul_Grand_Park-Gwacheon_Gyeonggi_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024866-d3164769-Reviews-or0-LetsRun_Park_Seoul_Seoul_Racecourse_Park-Gwacheon_Gyeonggi_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024866-d1868571-Reviews-or0-Seoul_Land-Gwacheon_Gyeonggi_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297886-d3387715-Reviews-or0-Daegu_Arboretum-Daegu.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297886-d10224627-Reviews-or0-Daegu_Samsung_Lions_Park-Daegu.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297886-d10841455-Reviews-or0-Suseongmot_Lake-Daegu.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297886-d2194163-Reviews-or0-Daegu_Duryu_Park-Daegu.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297886-d7108087-Reviews-or0-Kim_Gwangseok_Street-Daegu.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297886-d4095643-Reviews-or0-Apsan_Park-Daegu.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297886-d2099339-Reviews-or0-Seomun_Market-Daegu.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297886-d21294773-Reviews-or0-E_World-Daegu.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297893-d3694532-Reviews-or0-Daewangam_Park-Ulsan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297893-d3694399-Reviews-or0-Ulsan_Grand_Park-Ulsan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297893-d3694728-Reviews-or0-Taehwa_River_Seepri_Bamboo_Grove-Ulsan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297893-d3694531-Reviews-or0-Taehwagang_National_Garden-Ulsan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297893-d12499688-Reviews-or0-Ulsan_Bridge_Observatory-Ulsan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297893-d8285486-Reviews-or0-Yeomposan_Mountain-Ulsan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d1458553-Reviews-or0-Haeundae_Beach-Busan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d1605137-Reviews-or0-Gwangalli_Beach-Busan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d3901349-Reviews-or0-Busan_Gamcheon_Culture_Village-Busan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d1552284-Reviews-or0-Taejongdae-Busan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d7622552-Reviews-or0-Oryukdo_Skywalk-Busan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d21302089-Reviews-or0-Haedong_Yonggungsa-Busan.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d4731329-Reviews-or0-Historic_Park_of_Geoje_POW_Camp-Geoje_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d9570003-Reviews-or0-Windy_Hill-Geoje_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d3694606-Reviews-or0-Sinseondae-Geoje_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d5924172-Reviews-or0-Oedo_Botania-Geoje_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d11434671-Reviews-or0-Hakdong_Mongdol_Beach-Geoje_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g844468-d2276389-Reviews-or0-Maemuldo-Tongyeong_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g844468-d10793485-Reviews-or0-Skyline_Luge_Tongyeong-Tongyeong_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g844468-d7954545-Reviews-or0-Mireuksan_Mountain-Tongyeong_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g844468-d5073556-Reviews-or0-Hallyeo_National_Marine_Park_View_Ropeway-Tongyeong_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g844468-d4505779-Reviews-or0-Dongpirang_Village-Tongyeong_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d2190771-Reviews-or0-Odongdo-Yeosu_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d3164609-Reviews-or0-Hyangiram_Hermitage-Yeosu_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d8610894-Reviews-or0-Yeosu_Maritime_Cable_Car-Yeosu_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d3164604-Reviews-or0-Dolsan_Park-Yeosu_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024841-d12118160-Reviews-or0-Jindo_Tower-Jindo_gun_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024841-d3805581-Reviews-or0-Ullimsanbang_House-Jindo_gun_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024841-d10020518-Reviews-or0-Jindo_Miracle_Sea_Road_Festival-Jindo_gun_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297885-d2204891-Reviews-or0-Spirited_Garden-Jeju_Jeju_Island.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g983296-d3231669-Reviews-or0-Jeju_Olle_Trail_Routes-Jeju_Island.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1643550-d554534-Reviews-or0-Everland-Yongin_Gyeonggi_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1872665-d2076201-Reviews-or0-DMZ-Paju_Gyeonggi_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g424960-d320368-Reviews-or0-Hwaseong_Fortress-Suwon_Gyeonggi_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d3938069-Reviews-or0-Gyeongpo_Beach-Gangneung_Gangwon_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d554528-Reviews-or0-Bukhansan_National_Park-Seoul.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1143545-d2224863-Reviews-or0-Jeonju_Hanok_Village-Jeonju_Jeollabuk_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d320359-Reviews-or0-Changdeokgung_Palace-Seoul.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024864-d3618503-Reviews-or0-The_Garden_of_Morning_Calm-Gapyeong_gun_Gyeonggi_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d325043-Reviews-or0-National_Museum_of_Korea-Seoul.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074107-d1903766-Reviews-or0-Suncheon_Bay_Wetland_Reserve-Suncheon_Jeollanam_do.html'
]

In [7]:
urls = [
#     'https://www.tripadvisor.co.kr/Attraction_Review-g946498-d2476641-Reviews-or0-Buyeo_National_Museum-Buyeo_gun_Chungcheongnam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g946498-d2476642-Reviews-or0-Buyeo_Jeongnimsaji_Temple-Buyeo_gun_Chungcheongnam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024831-d8591806-Reviews-or0-Geumseongsanseong_Fortress-Damyang_gun_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024831-d3805591-Reviews-or0-Korea_Bamboo_Museum-Damyang_gun_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024831-d9586413-Reviews-or0-Metasequoia_lined_Road-Damyang_gun_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024839-d3805551-Reviews-or0-Baekyangsa_Temple-Jangseong_gun_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074106-d13861577-Reviews-or0-Naju_National_Museum-Naju_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074106-d14135468-Reviews-or0-Naju_Geumseonggwan-Naju_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074106-d15054422-Reviews-or0-Bitgaram_Lake_Park_Observation_Deck-Naju_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d7939824-Reviews-or0-Maengjongjuk_Theme_Park-Geoje_Gyeongsangnam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d11434671-Reviews-or0-Hakdong_Mongdol_Beach-Geoje_Gyeongsangnam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d7939801-Reviews-or0-Gonggoji-Geoje_Gyeongsangnam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d16656506-Reviews-or0-Maemiseong-Geoje_Gyeongsangnam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d21294709-Reviews-or0-Geoje_Botanic_Garden_Jungle_Dome-Geoje_Gyeongsangnam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d3164603-Reviews-or0-Yi_Sun_Shin_Square-Yeosu_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d8610894-Reviews-or0-Yeosu_Maritime_Cable_Car-Yeosu_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d3164630-Reviews-or0-Yeosu_Fish_Market-Yeosu_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d2190771-Reviews-or0-Odongdo-Yeosu_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d14058170-Reviews-or0-Nangman_Pocha_Street_Cart_Bars-Yeosu_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d3164604-Reviews-or0-Dolsan_Park-Yeosu_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d10399720-Reviews-or0-Manseongri_Black_Sand_Beach-Yeosu_Jeollanam_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d6569894-Reviews-or0-Seongdong_Market-Gyeongju_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d4087335-Reviews-or0-Gyeongju_World_Culture_Expo_Park-Gyeongju_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d3808317-Reviews-or0-Gyerim_Forest-Gyeongju_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d1554895-Reviews-or0-Donggung_Palace_and_Wolji_Pond-Gyeongju_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d2093002-Reviews-or0-Gyeongju_National_Museum-Gyeongju_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d3805607-Reviews-or0-Gyeongju_Historic_Area-Gyeongju_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d7372649-Reviews-or0-Gyeongju_Gyochon_Traditional_Village-Gyeongju_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g676092-d1755852-Reviews-or0-Jukdo_Market-Pohang_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g676092-d2571669-Reviews-or0-Homigot_Sunrise_Plaza-Pohang_Gyeongsangbuk_do.html#Reviews-or0',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g676092-d10188576-Reviews-or0-Guryongpo_Modern_History_Museum-Pohang_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g676092-d12550709-Reviews-or0-Yeongildae_Beach-Pohang_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024859-d8472393-Reviews-or0-Seongryugul_Cave-Uljin_gun_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024859-d3805623-Reviews-or0-Mangyangjeong_Pavilion-Uljin_gun_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024859-d3805520-Reviews-or0-Bulyoungsa-Uljin_gun_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074139-d10769654-Reviews-or0-Yonghwa_Beach-Samcheok_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074139-d6208766-Reviews-or0-Jangho_Port-Samcheok_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074139-d5924557-Reviews-or0-Samcheok_Ocean_Railbike-Samcheok_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074139-d6208735-Reviews-or0-Jukseoru_Pavilion-Samcheok_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1960566-d3805538-Reviews-or0-Yeongnyeongneung-Yeoju_si_Gyeonggi_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024868-d10338816-Reviews-or0-Namhansanseong_Fortress-Gwangju_Gyeonggi_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024868-d13086582-Reviews-or0-Hwadam_Botanic_Garden-Gwangju_Gyeonggi_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1047895-d8592154-Reviews-or0-Chungju_World_Martial_Arts_Park-Chungju_Chungcheongbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1047895-d14007009-Reviews-or0-Suanbo_Cherry_Blossom_Road-Chungju_Chungcheongbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1047895-d4094582-Reviews-or0-Gyemyungsan_Nature_Recreation_Forest-Chungju_Chungcheongbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1047895-d19886339-Reviews-or0-Muhak_Market-Chungju_Chungcheongbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d3938069-Reviews-or0-Gyeongpo_Beach-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d1966431-Reviews-or0-Ojukheon-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d4608666-Reviews-or0-Jeongdongjin_Beach-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d12526841-Reviews-or0-Anmok_Beach-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d1966433-Reviews-or0-Gangneung_Seongyojang-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d11885477-Reviews-or0-Jeongdong_Simgok_Badabuchae_gil_Trail-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d13422061-Reviews-or0-Gangneung_Jungang_Market-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d2355154-Reviews-or0-Hourglass_Park-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d14780079-Reviews-or0-Jumunjin_Port-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d18454052-Reviews-or0-Yeongjin_Beach-Gangneung_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074141-d6208731-Reviews-or0-Mukho_Port-Donghae_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074141-d2355156-Reviews-or0-Mukho_Lighthouse_Park-Donghae_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074141-d3805638-Reviews-or0-Samhwasa_Temple-Donghae_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074141-d6208728-Reviews-or0-Mureung_Valley-Donghae_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074141-d6208734-Reviews-or0-Chuam_Beach-Donghae_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074141-d2355158-Reviews-or0-Chotdaebawi_Candlestick_Rock-Donghae_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g1074141-d6208732-Reviews-or0-Cheongok_Cave-Donghae_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024862-d3805533-Reviews-or0-Juwangsan_National_Park-Cheongsong_gun_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024862-d8317435-Reviews-or0-Jusanji_Pond-Cheongsong_gun_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024853-d7126269-Reviews-or0-Sunrise_Park-Yeongdeok_gun_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024853-d4094892-Reviews-or0-Samsa_Marine_Park-Yeongdeok_gun_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024853-d3805389-Reviews-or0-Goesiri_Traditional_Village-Yeongdeok_gun_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2024853-d14112220-Reviews-or0-Yeongdeok_Crab_Village-Yeongdeok_gun_Gyeongsangbuk_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g612375-d6208181-Reviews-or0-Naksan_Beach-Yangyang_gun_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g612375-d3805539-Reviews-or0-Naksansa_Temple-Yangyang_gun_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g612375-d2583441-Reviews-or0-Osaek_Carbonated_Hot_Springs-Yangyang_gun_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g612375-d6208674-Reviews-or0-Hajodae-Yangyang_gun_Gangwon_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g2092512-d3805401-Reviews-or0-Icheon_Ceramics_Village-Icheon_Gyeonggi_do.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d1718638-Reviews-or0-Gukje_Market-Busan.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d1372972-Reviews-or0-Jagalchi_Market-Busan.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d1632406-Reviews-or210-Yongdusan_Park-Busan.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d1465262-Reviews-or0-UN_Memorial_Cemetery-Busan.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d4354725-Reviews-or0-Haeundae_Dalmaji_gil_Road-Busan.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d1605555-Reviews-or0-BIFF_Square-Busan.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d4095638-Reviews-or0-Minrak_Waterside_Park-Busan.html',
# 'https://www.tripadvisor.co.kr/Attraction_Review-g297884-d2337247-Reviews-or0-Seomyeon_Food_Alley-Busan.html',
]

In [11]:
urls = [
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074108-d3164604-Reviews-or0-Dolsan_Park-Yeosu_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024841-d12118160-Reviews-or0-Jindo_Tower-Jindo_gun_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024831-d8591806-Reviews-or0-Geumseongsanseong_Fortress-Damyang_gun_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024839-d3805551-Reviews-or0-Baekyangsa_Temple-Jangseong_gun_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074106-d13861577-Reviews-or0-Naju_National_Museum-Naju_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074106-d14135468-Reviews-or0-Naju_Geumseonggwan-Naju_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074106-d15054422-Reviews-or0-Bitgaram_Lake_Park_Observation_Deck-Naju_Jeollanam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d7939801-Reviews-or0-Gonggoji-Geoje_Gyeongsangnam_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297888-d1315474-Reviews-or0-Yangdong_Folk_Village-Gyeongju_Gyeongsangbuk_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074139-d10769654-Reviews-or0-Yonghwa_Beach-Samcheok_Gangwon_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1074139-d5924557-Reviews-or0-Samcheok_Ocean_Railbike-Samcheok_Gangwon_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1047895-d14007009-Reviews-or0-Suanbo_Cherry_Blossom_Road-Chungju_Chungcheongbuk_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1047895-d4094582-Reviews-or0-Gyemyungsan_Nature_Recreation_Forest-Chungju_Chungcheongbuk_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g1047895-d19886339-Reviews-or0-Muhak_Market-Chungju_Chungcheongbuk_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024853-d3805389-Reviews-or0-Goesiri_Traditional_Village-Yeongdeok_gun_Gyeongsangbuk_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g2024853-d14112220-Reviews-or0-Yeongdeok_Crab_Village-Yeongdeok_gun_Gyeongsangbuk_do.html',
#     'https://www.tripadvisor.co.kr/Attraction_Review-g297893-d8285486-Reviews-or0-Yeomposan_Mountain-Ulsan.html',
]

In [62]:
urls = [
    'https://www.tripadvisor.co.kr/Attraction_Review-g2024841-d10020518-Reviews-or0-Jindo_Miracle_Sea_Road_Festival-Jindo_gun_Jeollanam_do.html',
    'https://www.tripadvisor.co.kr/Attraction_Review-g1510776-d16656506-Reviews-or0-Maemiseong-Geoje_Gyeongsangnam_do.html',
    'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d4412874-Reviews-or0-Jumunjin_Port-Gangneung_Gangwon_do.html',
    'https://www.tripadvisor.co.kr/Attraction_Review-g317126-d18454052-Reviews-or0-Yeongjin_Beach-Gangneung_Gangwon_do.html',
]

In [63]:
travel_records

,place_name,user,travel_date,point,review


In [66]:
for url in urls:
    driver.get(url)
    time.sleep(1)
    driver.implicitly_wait(5)
    html = driver.page_source
    soup = BeautifulSoup(html , 'html.parser')
    
    try:
        rev_num_str = ignore_text(soup.select_one('.biGQs .Ci').get_text().replace(',',''))
    except:
        rev_num_str = 'q q q 10'
        
    try:
        rev_num = int(rev_num_str.split(' ')[3])
    except:
        rev_num = 10
    print(rev_num)
    page_num = (rev_num//10)*10
    print(page_num)
    text = 'Reviews-or'
    place = soup.select_one('.nrbon .biGQs').get_text()
    
    for page in range(0, page_num+1, 10):
#         try:
        num_text = text + str(page)
        print(page)

        url2 = url.replace('Reviews-or0', num_text)
        print(url2)
        driver.get(url2)
        time.sleep(1)
        driver.implicitly_wait(5)
        html = driver.page_source
        soup = BeautifulSoup(html , 'html.parser')
        time.sleep(1)
        try:
            reviews = soup.select('.LbPSX ._c')
        except:
            continue
            
        for review in reviews:
            tmp_lst = []
            nonamenum = 1
            try:
                name = ignore_text(review.select_one('.mwPje .XExLl .zpDvc .biGQs._P.fiohW').get_text())
            except:
                name = 'noname'+ str(nonamenum)
                nonamenum += 1
            print(name)
            stars_str = review.select_one('.UctUV.d.H0')
            stars = float(stars_str.get('aria-label').split(' ')[3])
            print(stars)
            
            date_many = review.select('.TreSq .biGQs')
            try:
                travel_date_str = date_many[0]
                travel_date_str = ignore_text(travel_date_str.get_text().replace(' 작성', ''))
                datetime_format = "%Y년 %M월 %d일"
                travel_date = datetime.strptime(travel_date_str, datetime_format)
                print(travel_date)
            except:
                travel_date_str = date_many[1]
                travel_date_str = ignore_text(travel_date_str.get_text().replace(' 작성', ''))
                datetime_format = "%Y년 %M월 %d일"
                travel_date = datetime.strptime(travel_date_str, datetime_format)
                print(travel_date)
            
            rev = ignore_text(review.select_one('._T.FKffI .fIrGe .biGQs .yCeTE').get_text())
            print(rev)
            
            tmp_lst = [place, name, travel_date, stars, rev]
            tmp_df = pd.DataFrame(data=[tmp_lst], columns=travel_records.columns)
            travel_records = pd.concat([travel_records, tmp_df])
            
#         except:
#             continue

10
10
0
https://www.tripadvisor.co.kr/Attraction_Review-g2024841-d10020518-Reviews-or0-Jindo_Miracle_Sea_Road_Festival-Jindo_gun_Jeollanam_do.html
vlogg
5.0
2016-01-26 00:06:00
전남  진도군 축제에요.  조수간만의 차이로 인해 생기는 바닷길이라서 그런지 외국인 관광객들도 많이 보이네요.직접 이런 지구과학적인 현상을  보니 정말 신기하고 이색적인 경험이었어요.
Christina S
5.0
2018-01-31 00:05:00
해안 도로 완전히 나타나는 않았다 불구하고, 멋진 시간을 보냈습니다! 우리 Trazy(통해서) 강풍 때문에, 길 바로 불어 파도가 설명했다 예약을 추천합니다 함께 가이드. 나에게 의미가 엄청나게 바람이 때문입니다. 그래서 바람이, 실제로 일부 상인들이 천막 문을 닫고 그들의 상품을 멀리 날리지 않도록 끝났습니다.길 나타나는, 조개류 찾는 강변 근처 바위에 오르고 시간을 보냈습니다 기다리고 있습니다. 많은 한국인들은 전복, 미역 찾는 힘들고도, 가끔 문어.한국 전통 장례식, 흥미로운. 재현모습. 공급업체 및 음식 텐트 t, 전 세계의 소수 민족의 요리를 먹을 수 있는 국제 지역. 나는 비빔밥 그리고 소주 한 병.한 언급한 바닷길 내가 방문했을 때 전체 모양 하지는 않았지만 저는 괜찮았어요. 나는 아직도 일생일대의 상당히 일단 참여할, 이봐, 해변에서 가장 최악의 날 는 최고의 하루를 더? :)
Gregor T
2.0
2018-01-18 00:05:00
올해는 불가사의 한 바다 도로가 완전히 접근 할 수 없었기 때문에 우리는 완전히 걸을 수 없었습니다. 나는 그들이 날짜를 오산 한 것 같아. 축제는 좋았지 만
10
https://www.tripadvisor.co.kr/Attraction_Review-g2024841-d10020518-Reviews-or10-Jindo_Miracle_Sea_Road_Festiva

jcj808
5.0
2015-01-11 00:08:00
경포대, 안목 해변쪽 주변 횟집은 2명이서 대개 8~10만원 정도가 소요됩니다.그래서 주문진수산시장으로 갔습니다.광어, 도미, 오징어, 멍게 2개였나..해서 횟감 30000원할복비 5000원상차림비 5000원매운탕비 5000원 이렇게 들었네요,참고로 킹크랩은 한마리에 5~7만원쪄서 나오는 상차림이 15000원 정도였어요.훨씬 저렴하게 먹을 수 있어요!
완전몸짱
2.0
2015-01-06 00:05:00
대게를 파는것도 좋구요 음식을 파는것도 좋구요 하지만 대게집에서 무리하게 상품을 파느라 바가지를 씌우는 인상이 들어 너무 기분이 영... 그랬습니다
kangdbang
5.0
2018-01-07 00:07:00
강원도에서 가장 큰 수산시장. 굳이 회를 먹지 않고 구경만해도 좋다. 아침 시간 시장의 부산함과 활기참은 언제나 밝은 기운으로 가득하다. 강원 가장 큰 생선과 해산물 시장. 이 시장의 에너지 및 밝기 전체.
Vicsite
4.0
2015-01-07 00:11:00
걸어 다녀 보고 장소도 시장 경매에는 해당되지 않습니다. 말린 생선 가게가 줄지어 있는 메인 다음 걸어. 나는 할 수 없는 생각이 많은 비즈니스 호텔. 이 경우는 아주 흥미로운 곳은 근처에 있습니다. 그리고 음식도 맛있었습니다!
10
10
0
https://www.tripadvisor.co.kr/Attraction_Review-g317126-d18454052-Reviews-or0-Yeongjin_Beach-Gangneung_Gangwon_do.html
sung58
4.0
2020-01-10 00:03:00
강릉시 연곡면에 있는 영진해변은 영진항부터 주문진 신리천에 이르는 해변으로 카페와 횟집 등이 즐비하게 늘어서 있습니다. 커피나 식사를 하면서 바다를 감상하기에 더없이 좋은 곳입니다.
sdc0503
5.0
2020-01-18 00:02:00
원래는 강릉사람만 알던 작은 시골마을 이였는데방송에 방영되고나서 많은 관광객들이 찾아오네요~강릉에는 이

In [67]:
travel_records

,place_name,user,travel_date,point,review
0,진도 신비의바닷길축제,vlogg,2016-01-26 00:06:00,5.0,전남 진도군 축제에요. 조수간만의 차이로 인해 생기는 바닷길이라서 그런지 외국인...
0,진도 신비의바닷길축제,Christina S,2018-01-31 00:05:00,5.0,"해안 도로 완전히 나타나는 않았다 불구하고, 멋진 시간을 보냈습니다! 우리 Traz..."
0,진도 신비의바닷길축제,Gregor T,2018-01-18 00:05:00,2.0,올해는 불가사의 한 바다 도로가 완전히 접근 할 수 없었기 때문에 우리는 완전히 걸...
0,매미성,teamwhite,2021-01-06 00:07:00,4.0,"매미성은 2003년 태풍 매미 때문에 바닷가 경작지를 잃은 거제시 장목면 주민이,자..."
0,매미성,J.G.YUN,2019-01-15 00:12:00,5.0,2003년 태풍 매미로 경작지를 잃은 주민이 자연재해로 부터 농작물 피해를 방지하기...
0,매미성,hyun,2019-01-08 00:07:00,3.0,나름 소박하고 거제대교를 배경으로 괜찮은 사진을 건질 수 있읍니다. 불에 구워주는 ...
0,매미성,Soon Cho,2019-01-18 00:03:00,3.0,한사람의 작업으로 이러한 축조물을 만들었다는 것이 경탄할만하나 주변 정비가 덜 되어...
0,매미성,vlogg,2019-01-15 00:02:00,5.0,매스컴에 소개된곳이라 의외로 관광객이 많아요.몽돌이 있는 바다앞에 태풍 매미때 개인...
0,주문진항,김준호,2020-01-30 00:11:00,5.0,강릉에 오면 주문진에 꼭 들러요수산시장에 가면 다양하고 맛있는 해산물들을 저렴한 가...
0,주문진항,Luna,2020-01-06 00:01:00,5.0,주문진항에는 다양한 횟집과 건어물상점이 모여있습니다. 여러 곳을 다녀보고 가격을 비...


In [68]:
travel_records.place_name.unique()

array(['진도 신비의바닷길축제', '매미성', '주문진항', '영진해변'], dtype=object)

In [69]:
travel_records.reset_index(drop=True, inplace=True)
travel_records

,place_name,user,travel_date,point,review
0,진도 신비의바닷길축제,vlogg,2016-01-26 00:06:00,5.0,전남 진도군 축제에요. 조수간만의 차이로 인해 생기는 바닷길이라서 그런지 외국인...
1,진도 신비의바닷길축제,Christina S,2018-01-31 00:05:00,5.0,"해안 도로 완전히 나타나는 않았다 불구하고, 멋진 시간을 보냈습니다! 우리 Traz..."
2,진도 신비의바닷길축제,Gregor T,2018-01-18 00:05:00,2.0,올해는 불가사의 한 바다 도로가 완전히 접근 할 수 없었기 때문에 우리는 완전히 걸...
3,매미성,teamwhite,2021-01-06 00:07:00,4.0,"매미성은 2003년 태풍 매미 때문에 바닷가 경작지를 잃은 거제시 장목면 주민이,자..."
4,매미성,J.G.YUN,2019-01-15 00:12:00,5.0,2003년 태풍 매미로 경작지를 잃은 주민이 자연재해로 부터 농작물 피해를 방지하기...
5,매미성,hyun,2019-01-08 00:07:00,3.0,나름 소박하고 거제대교를 배경으로 괜찮은 사진을 건질 수 있읍니다. 불에 구워주는 ...
6,매미성,Soon Cho,2019-01-18 00:03:00,3.0,한사람의 작업으로 이러한 축조물을 만들었다는 것이 경탄할만하나 주변 정비가 덜 되어...
7,매미성,vlogg,2019-01-15 00:02:00,5.0,매스컴에 소개된곳이라 의외로 관광객이 많아요.몽돌이 있는 바다앞에 태풍 매미때 개인...
8,주문진항,김준호,2020-01-30 00:11:00,5.0,강릉에 오면 주문진에 꼭 들러요수산시장에 가면 다양하고 맛있는 해산물들을 저렴한 가...
9,주문진항,Luna,2020-01-06 00:01:00,5.0,주문진항에는 다양한 횟집과 건어물상점이 모여있습니다. 여러 곳을 다녀보고 가격을 비...


In [70]:
travel_records2 = travel_records.copy()

In [71]:
# for i in range(0, 6218):
#     travel_records2['place'][i] = ignore_text(travel_records2['place'][i])
#     travel_records2['user'][i] = ignore_text(travel_records2['user'][i])
#     travel_records2['review'][i] = ignore_text(travel_records2['review'][i])

In [72]:
#travel_records.to_csv('./travel_records2-1.csv', encoding='utf-8-sig', index=False)

## concat

In [73]:
tr1 = pd.read_csv('./travel_records_con_0917.csv', encoding='utf-8-sig')

con = pd.concat([tr1, travel_records2])
con

,place_id,place_name,userid,user,travel_date,point,review
0,54.0,불국사,1138.0,Chuuk K,2022.1.2 0:02,4.0,정말 정말 오랜만에 경주 불국사를 설날 아침에 방문하였다 입장료가 인당 6000원...
1,54.0,불국사,4286.0,Oh!,2021.1.5 0:12,5.0,독특한 탑 양식을 보여주는 석가탑과 다보탑을 보는 것만으로도 충분한 가볼만합니다. ...
2,54.0,불국사,10269.0,병섭 김,2021.1.9 0:11,4.0,경주하면 불국사라할 정도로... 유명한 사찰이며 관광지입니다. 가이드 투어로 불국...
3,54.0,불국사,6667.0,composuretour,2021.1.16 0:03,5.0,부족한 배경지식에만 의존하지 말고또 인터넷 검색의 단편적인 정보에 의존하지 말고문화...
4,54.0,불국사,9243.0,ultraus,2020.1.22 0:10,5.0,경치가 아주 아름다움. 전시되어 있는 사리와 다보탑 석가탑들도 신비함을 자아내지만그...
...,...,...,...,...,...,...,...
51,NaN,주문진항,NaN,Vicsite,2015-01-07 00:11:00,4.0,걸어 다녀 보고 장소도 시장 경매에는 해당되지 않습니다. 말린 생선 가게가 줄지어 ...
52,NaN,영진해변,NaN,sung58,2020-01-10 00:03:00,4.0,강릉시 연곡면에 있는 영진해변은 영진항부터 주문진 신리천에 이르는 해변으로 카페와 ...
53,NaN,영진해변,NaN,sdc0503,2020-01-18 00:02:00,5.0,원래는 강릉사람만 알던 작은 시골마을 이였는데방송에 방영되고나서 많은 관광객들이 찾...
54,NaN,영진해변,NaN,상우 박,2019-01-30 00:11:00,4.0,누구나 도깨비의 명장면과 배경에 관심을 갖는다. 줄서서 기다리다 찍던 곳인데 그 주...


In [74]:
con.reset_index(drop=True, inplace=True)
con

,place_id,place_name,userid,user,travel_date,point,review
0,54.0,불국사,1138.0,Chuuk K,2022.1.2 0:02,4.0,정말 정말 오랜만에 경주 불국사를 설날 아침에 방문하였다 입장료가 인당 6000원...
1,54.0,불국사,4286.0,Oh!,2021.1.5 0:12,5.0,독특한 탑 양식을 보여주는 석가탑과 다보탑을 보는 것만으로도 충분한 가볼만합니다. ...
2,54.0,불국사,10269.0,병섭 김,2021.1.9 0:11,4.0,경주하면 불국사라할 정도로... 유명한 사찰이며 관광지입니다. 가이드 투어로 불국...
3,54.0,불국사,6667.0,composuretour,2021.1.16 0:03,5.0,부족한 배경지식에만 의존하지 말고또 인터넷 검색의 단편적인 정보에 의존하지 말고문화...
4,54.0,불국사,9243.0,ultraus,2020.1.22 0:10,5.0,경치가 아주 아름다움. 전시되어 있는 사리와 다보탑 석가탑들도 신비함을 자아내지만그...
...,...,...,...,...,...,...,...
22869,NaN,주문진항,NaN,Vicsite,2015-01-07 00:11:00,4.0,걸어 다녀 보고 장소도 시장 경매에는 해당되지 않습니다. 말린 생선 가게가 줄지어 ...
22870,NaN,영진해변,NaN,sung58,2020-01-10 00:03:00,4.0,강릉시 연곡면에 있는 영진해변은 영진항부터 주문진 신리천에 이르는 해변으로 카페와 ...
22871,NaN,영진해변,NaN,sdc0503,2020-01-18 00:02:00,5.0,원래는 강릉사람만 알던 작은 시골마을 이였는데방송에 방영되고나서 많은 관광객들이 찾...
22872,NaN,영진해변,NaN,상우 박,2019-01-30 00:11:00,4.0,누구나 도깨비의 명장면과 배경에 관심을 갖는다. 줄서서 기다리다 찍던 곳인데 그 주...


In [80]:
D = {1:['백마강'],
    2:['성흥산성'],
    3:['미암사'],
    4:['부소산성'],
    5:['백제문화단지'],
    6:['궁남지'],
    7:['오천항'],
    8:['오서산 자연 휴양림'],
    9:['군헌 갯벌 체험장'],
    10:['고대도'],
    11:['보령 노을 광장'],
    12:['청보리밭 폐목장'],
    13:['정혜사'],
    14:['칠갑산 도립 공원'],
    15:['청양 알프스 마을'],
    16:['홍천 동키 마을'],
    17:['홍천 은행 나무 숲'],
    18:['홍천 중앙시장'],
    19:['홍천 향교'],
    20:['척야산 문화 수목원'],
    21:['안흥찐빵 모락모락마을'],
    22:['풍수원 성당'],
    23:['청태산 자연휴양림'],
    24:['태기산'],
    25:['횡성 호수길'],
    26:['영월 중앙시장'],
    27:['선돌'],
    28:['라디오스타 박물관'],
    29:['청령포'],
    30:['문경새재도립공원'],
    31:['경체정'],
    32:['문경 전통시장'],
    33:['문경 오미자 테마 터널'],
    34:['대승사'],
    35:['황학산 수목원'],
    36:['강천섬'],
    37:['영월루'],
    38:['여주 현충탑'],
    39:['신륵사'],
    40:['용문산 관광단지'],
    41:['수풀로 양수리 수변공원'],
    42:['신안 철새 박물관'],
    43:['퍼플섬(반월도 매표소)'],
    44:['퍼플섬(박지도 매표소)'],
    45:['천사섬분재공원'],
    46:['1004뮤지엄파크'],
    47:['불갑저수지 수변공원'],
    48:['영광칠산타워'],
    49:['계마항'],
    50:['백마위해변'],
    51:['가마미 해수욕장'],
    52:['불갑사'],
    53:['정암루'],
    54:['불국사'],
    55:['경복궁'],
    56:['세량제'],
    57:['안압지'],
    58:['협재 해수욕장'],
    59:['청도 소싸움 경기장'],
    60:['대관령 양떼목장'],
    61:['보성 녹차밭 대한 다원'],
    62:['여좌천'],
    63:['낙안읍성 민속마을'],
    64:['담양 죽녹원'],
    65:['지리산 천왕봉'],
    66:['해인사'],
    67:['우도'],
    68:['섭지코지'],
    69:['우포늪'],
    70:['경화역 벚꽃길'],
    71:['광안대교'],
    72:['꽃지해수욕장'],
    73:['성산 일출봉'],
    74:['국립 과천 과학관'],
    75:['서울대공원'],
    76:['서울 경마 공원'],
    77:['서울랜드'],
    78:['대구수목원'],
    79:['대구삼성라이온즈파크'],
    80:['수성못'],
    81:['대구 두류 공원'],
    82:['김광석 길'],
    83:['앞산공원'],
    84:['서문 시장'],
    85:['이월드'],
    86:['대왕암 공원'],
    87:['울산 대공원'],
    88:['태화강 십리대밭'],
    89:['태화강 국가정원'],
    90:['울산대교 전망대'],
    91:['염포산'],
    92:['해운대'],
    93:['광안리해수욕장'],
    94:['감천 문화마을'],
    95:['태종대'],
    96:['오륙도 스카이워크'],
    97:['해동 용궁사'],
    98:['거제포로수용소유적공원'],
    99:['바람의언덕'],
    100:['신선대'],
    101:['외도 해상농원'],
    102:['학동몽돌해변'],
    103:['매물도'],
    104:['스카이라인 루지 통영'],
    105:['미륵산'],
    106:['한려수도 조망 케이블카'],
    107:['동피랑마을'],
    108:['오동도'],
    109:['향일암'],
#     110:['여수 해상케이블카'],
    111:['돌산 공원'],
    112:['진도타워'],
    113:['운림산방'],
    114:['진도 신비의바닷길축제'],
    115:['생각하는 정원'],
    116:['제주 올레길'],
    117:['에버랜드'],
    118:['DMZ'],
    119:['수원화성'],
    120:['경포해변'],
    121:['북한산 국립공원'],
    122:['전주 한옥 마을'],
    123:['창덕궁'],
    124:['아침 고요 수목원'],
    125:['국립중앙박물관'],
    126:['순천만 습지'],
    127:['국립 부여 박물관'],
    128:['부여 정림사지'],
    129:['금성산성'],
    130:['한국 대나무 박물관'],
    131:['담양 메타세쿼이아길'],
    132:['백양사'],
    133:['국립나주박물관'],
    134:['나주 금성관'],
    135:['빛가람호수공원전망대'],
    136:['맹종죽테마공원'],
#     137:['학동몽돌해별'],
    138:['공곶이'],
    139:['매미성'],
    140:['거제 식물원'],
    141:['이순신 광장'],
    142:['여수 해상케이블카'],
    143:['여수 수산시장'],
#     144:['오동도'],
    145:['낭만포차로'],
    146:['돌산 공원'],
    147:['만성리검은모래해변'],
    148:['양동 민속 마을'],
    149:['성동시장'],
    150:['경주세계문화엑스포공원'],
    151:['계림'],
    152:['국립경주박물관'],
    153:['경주역사유적지구'],
    154:['경주 교촌마을'],
    155:['죽도 시장'],
    156:['호미곶 해맞이 광장'],
    157:['구룡포근대역사거리'],
    158:['영일대해수욕장'],
    159:['성류굴'],
    160:['망양정'],
    161:['불영사'],
    162:['용화해수욕장'],
    163:['장호항'],
    164:['삼척 해양 레일바이크'],
    165:['죽서루'],
    166:['영녕릉'],
    167:['남한산성'],
    168:['화담숲'],
    169:['광주 한옥마을'],
    170:['충주 세계 무술공원'],
    171:['수안보 벚꽃길'],
    172:['계명산자연휴양림'],
    173:['무학시장'],
#     174:['경포해변'],
    175:['오죽헌'],
    176:['정동진'],
    177:['안목 해수욕장'],
    178:['강릉 선교장'],
    179:['정동심곡 바다부채길'],
    180:['강릉중앙시장'],
    181:['모래 시계 공원'],
    182:['주문진항'],
    183:['영진해변'],
    184:['묵호항'],
    185:['묵호등대 해양문화공간'],
    186:['삼화사'],
    187:['무릉계곡'],
    188:['추암해수욕장'],
    189:['촛대 바위'],
    190:['천곡천연동굴'],
    191:['주왕산국립공원'],
    192:['주산지'],
    193:['영덕 해맞이 공원'],
    194:['삼사해상공원'],
    195:['괴시리 전통마을'],
    196:['영덕대게마을'],
    197:['낙산 해수욕장'],
    198:['낙산사'],
    199:['양양 서피비치'],
    200:['오색 탄산온천'],
    201:['하조대'],
    202:['하조대해수욕장'],
    203:['이천도예마을'],
    204:['국제시장'],
    205:['자갈치 시장'],
    206:['용두산 공원'],
    207:['UN기념공원'],
    208:['해운대 달맞이길'],
    209:['BIFF 광장'],
    210:['민락수변공원'],
    211:['서면 먹자 골목'],
    }

In [81]:
# D = {"82":[0,2,12,15,25], "83":[56], "84":[30,31]} 
con['place_id'] = con.place_name.map({item: k for k, v in D.items() for item in v})

# d = {x: k for k, v in D.items() for x in v}
# con['place_id'] = con['place'].map(d)
# print (con)

In [82]:
# con.drop(columns=['place_id'], inplace=True)
con

,place_id,place_name,userid,user,travel_date,point,review
0,54,불국사,1138.0,Chuuk K,2022.1.2 0:02,4.0,정말 정말 오랜만에 경주 불국사를 설날 아침에 방문하였다 입장료가 인당 6000원...
1,54,불국사,4286.0,Oh!,2021.1.5 0:12,5.0,독특한 탑 양식을 보여주는 석가탑과 다보탑을 보는 것만으로도 충분한 가볼만합니다. ...
2,54,불국사,10269.0,병섭 김,2021.1.9 0:11,4.0,경주하면 불국사라할 정도로... 유명한 사찰이며 관광지입니다. 가이드 투어로 불국...
3,54,불국사,6667.0,composuretour,2021.1.16 0:03,5.0,부족한 배경지식에만 의존하지 말고또 인터넷 검색의 단편적인 정보에 의존하지 말고문화...
4,54,불국사,9243.0,ultraus,2020.1.22 0:10,5.0,경치가 아주 아름다움. 전시되어 있는 사리와 다보탑 석가탑들도 신비함을 자아내지만그...
...,...,...,...,...,...,...,...
22869,182,주문진항,NaN,Vicsite,2015-01-07 00:11:00,4.0,걸어 다녀 보고 장소도 시장 경매에는 해당되지 않습니다. 말린 생선 가게가 줄지어 ...
22870,183,영진해변,NaN,sung58,2020-01-10 00:03:00,4.0,강릉시 연곡면에 있는 영진해변은 영진항부터 주문진 신리천에 이르는 해변으로 카페와 ...
22871,183,영진해변,NaN,sdc0503,2020-01-18 00:02:00,5.0,원래는 강릉사람만 알던 작은 시골마을 이였는데방송에 방영되고나서 많은 관광객들이 찾...
22872,183,영진해변,NaN,상우 박,2019-01-30 00:11:00,4.0,누구나 도깨비의 명장면과 배경에 관심을 갖는다. 줄서서 기다리다 찍던 곳인데 그 주...


In [83]:
con.isnull().sum()

place_id         0
place_name       0
userid          56
user             0
travel_date      0
point            0
review         375
dtype: int64

In [84]:
con[con['place_id'].isnull()]

,place_id,place_name,userid,user,travel_date,point,review


In [85]:
#con['place_name'].unique()

In [86]:
con = con[['place_id', 'place_name','userid','user','travel_date','point','review']]

In [87]:
con2 = con.copy()

In [88]:
con2

,place_id,place_name,userid,user,travel_date,point,review
0,54,불국사,1138.0,Chuuk K,2022.1.2 0:02,4.0,정말 정말 오랜만에 경주 불국사를 설날 아침에 방문하였다 입장료가 인당 6000원...
1,54,불국사,4286.0,Oh!,2021.1.5 0:12,5.0,독특한 탑 양식을 보여주는 석가탑과 다보탑을 보는 것만으로도 충분한 가볼만합니다. ...
2,54,불국사,10269.0,병섭 김,2021.1.9 0:11,4.0,경주하면 불국사라할 정도로... 유명한 사찰이며 관광지입니다. 가이드 투어로 불국...
3,54,불국사,6667.0,composuretour,2021.1.16 0:03,5.0,부족한 배경지식에만 의존하지 말고또 인터넷 검색의 단편적인 정보에 의존하지 말고문화...
4,54,불국사,9243.0,ultraus,2020.1.22 0:10,5.0,경치가 아주 아름다움. 전시되어 있는 사리와 다보탑 석가탑들도 신비함을 자아내지만그...
...,...,...,...,...,...,...,...
22869,182,주문진항,NaN,Vicsite,2015-01-07 00:11:00,4.0,걸어 다녀 보고 장소도 시장 경매에는 해당되지 않습니다. 말린 생선 가게가 줄지어 ...
22870,183,영진해변,NaN,sung58,2020-01-10 00:03:00,4.0,강릉시 연곡면에 있는 영진해변은 영진항부터 주문진 신리천에 이르는 해변으로 카페와 ...
22871,183,영진해변,NaN,sdc0503,2020-01-18 00:02:00,5.0,원래는 강릉사람만 알던 작은 시골마을 이였는데방송에 방영되고나서 많은 관광객들이 찾...
22872,183,영진해변,NaN,상우 박,2019-01-30 00:11:00,4.0,누구나 도깨비의 명장면과 배경에 관심을 갖는다. 줄서서 기다리다 찍던 곳인데 그 주...


In [89]:
con2['userid'] = con2.groupby(['user']).ngroup()
con2

,place_id,place_name,userid,user,travel_date,point,review
0,54,불국사,1138,Chuuk K,2022.1.2 0:02,4.0,정말 정말 오랜만에 경주 불국사를 설날 아침에 방문하였다 입장료가 인당 6000원...
1,54,불국사,4287,Oh!,2021.1.5 0:12,5.0,독특한 탑 양식을 보여주는 석가탑과 다보탑을 보는 것만으로도 충분한 가볼만합니다. ...
2,54,불국사,10275,병섭 김,2021.1.9 0:11,4.0,경주하면 불국사라할 정도로... 유명한 사찰이며 관광지입니다. 가이드 투어로 불국...
3,54,불국사,6671,composuretour,2021.1.16 0:03,5.0,부족한 배경지식에만 의존하지 말고또 인터넷 검색의 단편적인 정보에 의존하지 말고문화...
4,54,불국사,9248,ultraus,2020.1.22 0:10,5.0,경치가 아주 아름다움. 전시되어 있는 사리와 다보탑 석가탑들도 신비함을 자아내지만그...
...,...,...,...,...,...,...,...
22869,182,주문진항,5811,Vicsite,2015-01-07 00:11:00,4.0,걸어 다녀 보고 장소도 시장 경매에는 해당되지 않습니다. 말린 생선 가게가 줄지어 ...
22870,183,영진해변,9020,sung58,2020-01-10 00:03:00,4.0,강릉시 연곡면에 있는 영진해변은 영진항부터 주문진 신리천에 이르는 해변으로 카페와 ...
22871,183,영진해변,8744,sdc0503,2020-01-18 00:02:00,5.0,원래는 강릉사람만 알던 작은 시골마을 이였는데방송에 방영되고나서 많은 관광객들이 찾...
22872,183,영진해변,10367,상우 박,2019-01-30 00:11:00,4.0,누구나 도깨비의 명장면과 배경에 관심을 갖는다. 줄서서 기다리다 찍던 곳인데 그 주...


In [90]:
con2 = con2[['place_id', 'place_name','userid','user','travel_date','point','review']]
con2

,place_id,place_name,userid,user,travel_date,point,review
0,54,불국사,1138,Chuuk K,2022.1.2 0:02,4.0,정말 정말 오랜만에 경주 불국사를 설날 아침에 방문하였다 입장료가 인당 6000원...
1,54,불국사,4287,Oh!,2021.1.5 0:12,5.0,독특한 탑 양식을 보여주는 석가탑과 다보탑을 보는 것만으로도 충분한 가볼만합니다. ...
2,54,불국사,10275,병섭 김,2021.1.9 0:11,4.0,경주하면 불국사라할 정도로... 유명한 사찰이며 관광지입니다. 가이드 투어로 불국...
3,54,불국사,6671,composuretour,2021.1.16 0:03,5.0,부족한 배경지식에만 의존하지 말고또 인터넷 검색의 단편적인 정보에 의존하지 말고문화...
4,54,불국사,9248,ultraus,2020.1.22 0:10,5.0,경치가 아주 아름다움. 전시되어 있는 사리와 다보탑 석가탑들도 신비함을 자아내지만그...
...,...,...,...,...,...,...,...
22869,182,주문진항,5811,Vicsite,2015-01-07 00:11:00,4.0,걸어 다녀 보고 장소도 시장 경매에는 해당되지 않습니다. 말린 생선 가게가 줄지어 ...
22870,183,영진해변,9020,sung58,2020-01-10 00:03:00,4.0,강릉시 연곡면에 있는 영진해변은 영진항부터 주문진 신리천에 이르는 해변으로 카페와 ...
22871,183,영진해변,8744,sdc0503,2020-01-18 00:02:00,5.0,원래는 강릉사람만 알던 작은 시골마을 이였는데방송에 방영되고나서 많은 관광객들이 찾...
22872,183,영진해변,10367,상우 박,2019-01-30 00:11:00,4.0,누구나 도깨비의 명장면과 배경에 관심을 갖는다. 줄서서 기다리다 찍던 곳인데 그 주...


In [91]:
len(con2.place_id.unique())

190

In [92]:
con2.to_csv('./travel_records_con_0917.csv', encoding='utf-8-sig', index=False)